In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.layers import GlobalAveragePooling1D, Dense, Input
import transformers 
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

/tmp/ipykernel_28954/2685073530.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-03 20:01:12.173351: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 20:01:12.173424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 20:01:12.174517: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable 

In [2]:
train = pd.read_csv("train.csv",usecols=[3,4])

In [3]:
train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
def prepro(t):
    t = t.lower()
    t = re.sub(r"[^a-zA-Z\s]","",t)
    t = re.sub(r"won\'t", " will not", t)
    t = re.sub(r"won\'t've", " will not have", t)
    t = re.sub(r"can\'t", " can not", t)
    t = re.sub(r"don\'t", " do not", t)
    t = re.sub(r"can\'t've", " can not have", t)
    t = re.sub(r"ma\'am", " madam", t)
    t = re.sub(r"let\'s", " let us", t)
    t = re.sub(r"ain\'t", " am not", t)
    t = re.sub(r"shan\'t", " shall not", t)
    t = re.sub(r"sha\n't", " shall not", t)
    t = re.sub(r"o\'clock", " of the clock", t)
    t = re.sub(r"y\'all", " you all", t)
    t = re.sub(r"n\'t", " not", t)
    t = re.sub(r"n\'t've", " not have", t)
    t = re.sub(r"\'re", " are", t)
    t = re.sub(r"\'s", " is", t)
    t = re.sub(r"\'d", " would", t)
    t = re.sub(r"\'d've", " would have", t)
    t = re.sub(r"\'ll", " will", t)
    t = re.sub(r"\'ll've", " will have", t)
    t = re.sub(r"\'t", " not", t)
    t = re.sub(r"\'ve", " have", t)
    t = re.sub(r"\'m", " am", t)
    t = re.sub(r"\'re", " are", t)
    t = " ".join(re.findall(r"[^\W\d_]+|\d+",t))
    t = re.sub("\s+"," ", t)

    processed = tokenizer(
    t,
    max_length=128,
    padding="max_length",
    truncation = True,
    return_attention_mask=True   
    )
    input_ids = np.array(processed["input_ids"])
    attention_mask = np.array(processed["attention_mask"])
    return {"input_ids": input_ids, "attention_mask": attention_mask}

In [6]:
train["text"] = train["text"].apply(prepro)

In [7]:
train.head()["text"][0]["input_ids"]

array([  101,  2256, 15616,  2024,  1996,  3114,  1997,  2023,  8372,
        2089, 16455,  9641,  2149,  2035,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [8]:
train.head()["text"][0]["attention_mask"]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
Xtra, Xtes, Ytra, Ytes = train_test_split(train["text"], train["target"],test_size=0.15)

In [10]:
def seperator(x):
    input_ids = []
    attention_masks = []
    for i in x.values:
        input_ids.append(i["input_ids"])
        attention_masks.append(i["attention_mask"])
    a = {"inp":input_ids,"att":attention_masks}
    return pd.DataFrame(a)

In [11]:
Xtra = seperator(Xtra)
Xtes = seperator(Xtes)

In [12]:
max_len = 128

In [17]:
model_ids  = Input(shape=(max_len, ), dtype = tf.int32)
model_mask = Input(shape=(max_len, ), dtype = tf.int32)

bert_model = TFBertModel.from_pretrained('bert-base-uncased')
x = bert_model(input_ids = model_ids, attention_mask = model_mask)
x = tf.keras.layers.GlobalAveragePooling1D()(x.last_hidden_state)    
outputs = Dense(1)(x)

model = tf.keras.Model(inputs = (model_ids, model_mask), outputs = outputs)

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = "binary_crossentropy",
    metrics=[tf.keras.metrics.BinaryAccuracy()])
model.fit((np.asarray(Xtra["inp"].values),np.asarray(Xtra["att"].values)),Xtes,verbose=1,epochs=10)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).